In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import joblib
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score

In [2]:

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # resize images
    transforms.ToTensor(),           # convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(root='/Users/zachary/Documents/UCI/Grad/F2025/CS284A/Project/aptos2019-blindness-detection', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [3]:
!pip install pandas

In [4]:
import pandas as pd
from torch.utils.data import Dataset, dataloader
from PIL import Image
from torchvision import transforms 
import os



In [5]:
class DRDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 1]
        img_path = os.path.join(self.img_dir,f'{img_name}.png')
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label



In [ ]:
train_img_path = "../aptos2019-blindness-detection/train_images"
train_csv_path = "../aptos2019-blindness-detection/train.csv"

test_img_path = "../aptos2019-blindness-detection/test_images"
test_csv_path = "../aptos2019-blindness-detection/test.csv"

In [7]:
transform = transforms.Compose([
    transforms.Resize((480, 480)),
    transforms.ToTensor()
])

train_dataset = DRDataset(
    csv_file=train_csv_path,
    img_dir=train_img_path,
    transform=transform
)
batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = DRDataset(
    csv_file= test_csv_path,
    img_dir= test_img_path,
    transform=transform
)
batch_size = 100
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [8]:
print('Train data: ', len(train_dataset))
print('Test data: ', len(test_dataset))

Train data:  3662
Test data:  1928


In [9]:
X,y = next(iter(train_loader))

In [10]:
X.shape, y.shape

(torch.Size([100, 3, 480, 480]), torch.Size([100]))

In [11]:
for images, labels in train_loader:
    print(images.shape)
    print(labels)

KeyboardInterrupt: 

In [22]:
class DRConvNet(nn.Module):
    def __init__(self, num_classes = 5):
        super(DRConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size = 3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size = 3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        ## 480 * 480

        final_size = 58
        self.fc1 = nn.Linear(128 * final_size**2 , num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(.5)
    def forward(self, x):
        print(X.shape)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.pool(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        return x

In [23]:
dr_conv_model = DRConvNet()

In [24]:
num_epochs = 50
learning_rate = 0.01
optimizer = torch.optim.Adam(dr_conv_model.parameters(), learning_rate)

In [25]:
criterion = nn.CrossEntropyLoss()

In [27]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (X, y) in enumerate(train_loader):
        # images = images.to(device)
        # labels = labels.to(device)

        # Forward pass
        
        output = dr_conv_model(X)
        
        loss = criterion(output, y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


KeyboardInterrupt: 